In [1]:
import subprocess
import datetime
import os

import pandas as pd
from datetime import datetime
import timeout_decorator


@timeout_decorator.timeout(10*60) # seconds
def gen_exec(cmd, check_return: bool = False):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code and check_return:
        raise subprocess.CalledProcessError(return_code, cmd)


def exec_bash(cmd: str, check_return: bool = False):
    return [s for s in gen_exec(cmd.split(), check_return)]
    
def exec_bash_print(cmd: str, check_return: bool = False):
    for path in gen_exec(cmd.split(), check_return):
        print(path, end="")
        
def strtime():
    return datetime.now().strftime("%y%m%d_%H%M%S")



from pathlib import Path

prj_path = Path(str(Path("../../../").resolve())+"/")
exp_folder = Path(prj_path, "papers/neurips21/")
res_folder = Path(exp_folder, "output")
model_folder = Path(exp_folder, "models")

jar_file = Path(prj_path, "target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar")
javafile = Path(exp_folder, "Experiments.java")
#java = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home/bin/java"
java = "java"


print(prj_path)
print(exp_folder)
print(res_folder)
print(model_folder)

print(jar_file)

/root/dev/credici
/root/dev/credici/papers/neurips21
/root/dev/credici/papers/neurips21/output
/root/dev/credici/papers/neurips21/models
/root/dev/credici/target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar


In [2]:
#cmd= f"java -cp {conf.jar_file} {javafile} {cmd}"
#output = exec_bash(cmd)

In [3]:
def run(model, datasize = 1000, executions=20, logfile=None, output = None, datafile=None, seed = 0):
    logfile = logfile or Path(res_folder, f"{strtime()}_log.txt")
    output = output or res_folder
    output.mkdir(parents=True, exist_ok=True)
    #print(model)
    modelfile = Path(model_folder, model)
    params = f"--executions {executions} --datasize {datasize} --policy LAST --output {output} "\
        f"--logfile {logfile} --seed {seed} -q "
    
    if datafile is not None:
        params += f"-f {datafile}"
    
    params += f" {modelfile}"

    cmd = f"{java} -cp {jar_file} {javafile} {params}"
    #print(cmd)
    output = exec_bash(cmd)  
    #print(output)
    exec(output[0])
    return locals()["results"]

#results = run(model = "set1/chain_twExo1_nEndo4_6.uai", datasize=500)

In [4]:
######### 
## Modify if needed:
modelset="seq0"
datasize = 1000
#########

stime = strtime()

# Log file and output folder
logfile = Path(res_folder, f"{stime}_log.txt")
output_folder = Path(res_folder, stime)

# Get the models
models = [Path(modelset, file) for file in os.listdir(Path(model_folder,modelset)) if file.endswith(".uai")]
datafiles = [Path(modelset, file) for file in os.listdir(Path(model_folder,modelset)) if file.endswith(".csv")]

print(logfile)
print(output_folder)
print(len(models))
print(len(datafiles))

m = models[0]
df = Path(str(m).replace(".uai",".csv"))
df in datafiles

/root/dev/credici/papers/neurips21/output/210520_194904_log.txt
/root/dev/credici/papers/neurips21/output/210520_194904
60
60


True

In [5]:
######### 
## Modify if needed:
start = 0
#########

blacklist = []
res_dicts = []

for i,m in enumerate(models[start:]):
    print(f"{i+start}/{len(models)}: {m}")
    #print(i not in blacklist)
    if i+start not in blacklist:
        try:            
            kwargs = dict(model = m, datasize=datasize, logfile=logfile, output=output_folder)
            df = Path(str(m).replace(".uai",".csv"))
            if df in datafiles:
                kwargs["datafile"] = Path(model_folder, df)             
            res_dicts.append(run(**kwargs))
        except timeout_decorator.TimeoutError:
            print("timeout")
    
print("finished")

0/60: seq0/chain_twExo0_nEndo15_8.uai


KeyboardInterrupt: 

In [ ]:
!tail -f /root/dev/credici/papers/neurips21/output/210520_194904_log.txt


'set1_chain_twExo0_nEndo6_16.uai_x20_d1000_m200.py'

In [43]:
!java -cp /root/dev/credici/target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar /root/dev/credici/papers/neurips21/Experiments.java --executions 20 --datasize 1000 --policy LAST --output /root/dev/credici/papers/neurips21/output/210517_161626 --logfile /root/dev/credici/papers/neurips21/output/210517_161626_log.txt --seed 0 -t 3 /root/dev/credici/papers/neurips21/models/set1/chain_twExo3_nEndo6_6.uai

Note: /root/dev/credici/papers/neurips21/Experiments.java uses unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.
[2021-05-17T18:26:29.406243][INFO][java] Set up log file: /root/dev/credici/papers/neurips21/output/210517_161626_log.txt
[2021-05-17T18:26:29.466695][INFO][java] _x20_pLAST_d1000_m200_s0
[2021-05-17T18:26:29.470234][INFO][java] Input args: --executions;20;--datasize;1000;--policy;LAST;--output;/root/dev/credici/papers/neurips21/output/210517_161626;--logfile;/root/dev/credici/papers/neurips21/output/210517_161626_log.txt;--seed;0;-t;3;/root/dev/credici/papers/neurips21/models/set1/chain_twExo3_nEndo6_6.uai
[2021-05-17T18:26:29.474513][INFO][java] Reading model at /root/dev/credici/papers/neurips21/models/set1/chain_twExo3_nEndo6_6.uai
[2021-05-17T18:26:29.539285][INFO][java] Loaded SCM: ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [(6,0), (8,0), (0,1), (7,1), (8,1), (9,1), (1,2), (6,2), (7,2), (10,2), (2,3), (9,3), (3,4), (10,4), (4,5), (11,5)])

In [4]:
java -cp /root/dev/credici/target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar /root/dev/credici/papers/neurips21/SequentialModelGen.java




1000_1_5_true_1
0000000./papers/neurips21/models/tmp/chain_twExo1_nEndo5_1
Exception in thread "main" java.io.FileNotFoundException: ./papers/neurips21/models/tmp/chain_twExo1_nEndo5_1.uai (No such file or directory)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:292)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:235)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:124)
	at java.base/java.io.FileWriter.<init>(FileWriter.java:66)
	at ch.idsia.crema.model.io.uai.UAIWriter.initWriter(UAIWriter.java:46)
	at ch.idsia.credici.model.io.uai.CausalUAIWriter.<init>(CausalUAIWriter.java:49)
	at ch.idsia.credici.model.io.uai.CausalUAIWriter.write(CausalUAIWriter.java:32)
	at ch.idsia.credici.IO.writeUAI(IO.java:30)
	at ch.idsia.credici.IO.write(IO.java:49)
	at neurips21.SequentialModelGen.save(SequentialModelGen.java:37)
	at neurips21.SequentialModelGen.buildModel(SequentialMode

In [ ]:
!mvn clean compile assembly:single

In [ ]:
./experiments.py 0 seq0 > output0.out 2>&1 &
./experiments.py 0 seq11 > output1a.out 2>&1 &
./experiments.py 0 seq1b > output1b.out 2>&1 &
